<a href="https://colab.research.google.com/github/brijeshdankhara/word-sequence/blob/master/word_sequencer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#google drive dependecies
!pip install -U PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# data dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16,10]
import seaborn as sns

%matplotlib inline
plt.rcParams['axes.unicode_minus'] = False

Requirement already up-to-date: PyDrive in /usr/local/lib/python3.6/dist-packages (1.3.1)


In [0]:
# authenticate
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
rotten  = drive.CreateFile({'id': '1N8WCMci_jpDHwCVgSED-B9yts-q9_Bb5'})
rotten.GetContentFile('rotten.csv')

In [0]:
import nltk
import gensim.models.word2vec as w2v
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [0]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re

In [0]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


**Set up logging**

In [0]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

**Download NLTK tokenizer models **



In [0]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Prepare Corpus

Load rotten tomatoes csv file.

In [0]:
df = pd.read_csv('rotten.csv')
df.head()

,Freshness,Review
0,1,"Manakamana doesn't answer any questions, yet ..."
1,1,Wilfully offensive and powered by a chest-thu...
2,0,It would be difficult to imagine material mor...
3,0,Despite the gusto its star brings to the role...
4,0,If there was a good idea at the core of this ...


Make a corpus by combinging all the review.

In [0]:
def createCorpusFromReviews(s):
    result = ""
    for i in s:
        if (str(i).endswith('.')):
            result = result+i
        else:
            result = result+"."+i
    return result
    
corpus_raw = createCorpusFromReviews(df['Review'])

Split corpus into sentences

In [0]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [0]:
raw_sentences = tokenizer.tokenize(corpus_raw)

In [0]:
#convert into a list of words
#remove unnnecessary,, split into words, no hyphens
#list of words
def sentence_to_wordlist(raw):
    clean = re.sub("[^a-zA-Z]"," ", raw)
    words = clean.split()
    return words

In [0]:
#sentence where each word is tokenized
sentences = []
for raw_sentence in raw_sentences:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_wordlist(raw_sentence))

In [0]:
print(raw_sentences[5])
print(sentence_to_wordlist(raw_sentences[5]))

Gleeson goes the Hallmark Channel route, damaging an intermittently curious entry in the time travel subgenre.
['Gleeson', 'goes', 'the', 'Hallmark', 'Channel', 'route', 'damaging', 'an', 'intermittently', 'curious', 'entry', 'in', 'the', 'time', 'travel', 'subgenre']


In [0]:
token_count = sum([len(sentence) for sentence in sentences])
print("The book corpus contains {0:,} tokens".format(token_count))

The book corpus contains 10,877,927 tokens


In [0]:
len(sentences)

543222

##Train word2vec

In [0]:
#ONCE we have vectors
#step 3 - build model
#3 main tasks that vectors help with
#DISTANCE, SIMILARITY, RANKING

# Dimensionality of the resulting word vectors.
#more dimensions, more computationally expensive to train
#but also more accurate
#more dimensions = more generalized
num_features = 300
# Minimum word count threshold.
min_word_count = 3

# Number of threads to run in parallel.
#more workers, faster we train
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7

# Downsample setting for frequent words.
#0 - 1e-5 is good for this
downsampling = 1e-3

# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

In [0]:
thrones2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

In [0]:
thrones2vec.build_vocab(sentences)

2019-04-10 05:38:39,329 : INFO : collecting all words and their counts
2019-04-10 05:38:39,331 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-04-10 05:38:39,401 : INFO : PROGRESS: at sentence #10000, processed 200217 words, keeping 21805 word types
2019-04-10 05:38:39,474 : INFO : PROGRESS: at sentence #20000, processed 400868 words, keeping 30495 word types
2019-04-10 05:38:39,547 : INFO : PROGRESS: at sentence #30000, processed 602203 words, keeping 36396 word types
2019-04-10 05:38:39,613 : INFO : PROGRESS: at sentence #40000, processed 802513 words, keeping 41119 word types
2019-04-10 05:38:39,684 : INFO : PROGRESS: at sentence #50000, processed 1003891 words, keeping 45314 word types
2019-04-10 05:38:39,761 : INFO : PROGRESS: at sentence #60000, processed 1205193 words, keeping 48794 word types
2019-04-10 05:38:39,839 : INFO : PROGRESS: at sentence #70000, processed 1404797 words, keeping 51776 word types
2019-04-10 05:38:39,918 : INFO : PROGRESS: 

In [0]:
print("Word2Vec vocabulary length:", len(thrones2vec.wv.vocab))

Word2Vec vocabulary length: 61500


**Start training, this might take a minute or two...**

In [0]:
thrones2vec.train(sentences, total_examples=543222, epochs=10)

2019-04-10 05:48:07,405 : INFO : training model with 2 workers on 61500 vocabulary and 300 features, using sg=1 hs=0 sample=0.001 negative=5 window=7
2019-04-10 05:48:08,569 : INFO : EPOCH 1 - PROGRESS: at 1.01% examples, 74912 words/s, in_qsize 3, out_qsize 0
2019-04-10 05:48:09,739 : INFO : EPOCH 1 - PROGRESS: at 2.11% examples, 77368 words/s, in_qsize 3, out_qsize 0
2019-04-10 05:48:10,739 : INFO : EPOCH 1 - PROGRESS: at 3.02% examples, 77503 words/s, in_qsize 2, out_qsize 1
2019-04-10 05:48:11,740 : INFO : EPOCH 1 - PROGRESS: at 3.94% examples, 77520 words/s, in_qsize 3, out_qsize 0
2019-04-10 05:48:12,744 : INFO : EPOCH 1 - PROGRESS: at 4.85% examples, 77538 words/s, in_qsize 3, out_qsize 0
2019-04-10 05:48:13,755 : INFO : EPOCH 1 - PROGRESS: at 5.86% examples, 78622 words/s, in_qsize 3, out_qsize 0
2019-04-10 05:48:14,910 : INFO : EPOCH 1 - PROGRESS: at 6.87% examples, 77909 words/s, in_qsize 3, out_qsize 0
2019-04-10 05:48:15,930 : INFO : EPOCH 1 - PROGRESS: at 7.88% examples, 7

(84653726, 108779270)

In [0]:
if not os.path.exists("trained"):
    os.makedirs("trained")
    
thrones2vec.save(os.path.join("trained", "thrones2vec.w2v"))

2019-04-10 06:06:12,680 : INFO : saving Word2Vec object under trained/thrones2vec.w2v, separately None
2019-04-10 06:06:12,682 : INFO : storing np array 'vectors' to trained/thrones2vec.w2v.wv.vectors.npy
2019-04-10 06:06:12,774 : INFO : not storing attribute vectors_norm
2019-04-10 06:06:12,776 : INFO : storing np array 'syn1neg' to trained/thrones2vec.w2v.trainables.syn1neg.npy
2019-04-10 06:06:12,890 : INFO : not storing attribute cum_table
2019-04-10 06:06:13,136 : INFO : saved trained/thrones2vec.w2v


In [0]:
thrones2vec.most_similar('Interstellar')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('neg', 0.476460725069046),
 ('Inception', 0.46428507566452026),
 ('Prometheus', 0.44643887877464294),
 ('Reloaded', 0.4390377998352051),
 ('Nolan', 0.43732231855392456),
 ('Screamers', 0.43477416038513184),
 ('Machina', 0.4314234256744385),
 ('Krrish', 0.42766261100769043),
 ('Tastes', 0.42237749695777893),
 ('Cid', 0.42005953192710876)]